In [1]:
search_for = 139
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08853292465209961
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 376006121
type: [1, 1, 1, 1, 139] 139
cases of this type: 373301041
100000 11.254104412110076
200000 21.279559160442158
300000 21.25701382632681
400000 20.22616341279747
500000 20.439410430809005
600000 21.061449527064514
700000 20.31384485821252
800000 21.622659052194496
900000 21.05493137782177
1000000 23.537877443437036
1100000 19.113491085856886
1200000 21.766582320682165
1300000 20.801935338873985
1400000 6.278358988931281
1500000 19.9006463071606
1600000 21.06857804529091
1700000 26.29540778695367
1800000 20.76198929621914
1900000 22.91737163698107
2000000 26.41656060835355
2100000 23.76447773223083
2200000 30.082382988449503
2300000 34.5759659316841
2400000 45.3763645234202
2500000 45.38026473166221
2600000 45.34769634699325
2700000 19.51475400405317
2800000 31.20101958721755
2900000 57.49668792032594
3000000 58.25953848521

29900000 33.74295390927218
30000000 41.510503211794536
30100000 37.84706188871058
30200000 39.50562457391616
30300000 36.576230814167694
30400000 41.55196936753538
30500000 35.715775092137676
30600000 40.26950191595574
30700000 37.119526692739484
30800000 38.51957666941038
30900000 18.73568763866179
31000000 26.722076465785413
31100000 37.544860580773225
31200000 37.66971811960761
31300000 35.645426790092294
31400000 40.048836220414614
31500000 33.78977407389764
31600000 36.993686646456084
31700000 39.993668684940396
31800000 34.55129900268221
31900000 38.275318728365164
32000000 25.392335912259625
32100000 26.641154942654563
32200000 36.36149075349817
32300000 36.418771736169234
32400000 37.34089151722099
32500000 13.901657264759315
32600000 32.699821844084596
32700000 36.15711411295815
32800000 37.275677790352454
32900000 37.93010000380555
33000000 31.97793834469499
33100000 36.68951172262856
33200000 33.36894499737885
33300000 35.72436499121497
33400000 33.9585155039808
33500000 32.

59800000 31.80185432446531
59900000 30.762613963909903
60000000 29.820009502975797
60100000 30.5558326013359
60200000 28.767975379930903
60300000 29.436274008752676
60400000 24.393914604285314
60500000 11.974405670749823
60600000 28.08562904562007
60700000 29.326557702606685
60800000 28.370591949287032
60900000 30.01400553324243
61000000 29.63284197049643
61100000 29.806755817912467
61200000 29.41988123165555
61300000 29.989148456258675
61400000 13.071807812779394
61500000 32.52882517280257
61600000 29.610283262343984
61700000 31.84928083089414
61800000 30.83664775390752
61900000 33.41320573142679
62000000 32.28155998370126
62100000 33.598348065589896
62200000 27.74350522244211
62300000 15.461949369707476
62400000 31.998527001457248
62500000 31.23074805264229
62600000 31.592766560137743
62700000 33.36776683149816
62800000 30.19512131872496
62900000 29.122963009999754
63000000 29.17814125685759
63100000 20.824074097757634
63200000 16.097847139353775
63300000 28.998507987140723
63400000 

89600000 27.363962625291197
89700000 27.42874446360425
89800000 26.48901571261244
89900000 27.42568817724933
90000000 8.997657787043247
90100000 24.930875116549558
90200000 26.5433127432731
90300000 27.066001121667224
90400000 26.463464372808886
90500000 26.64106337141
90600000 26.705476618246784
90700000 11.158850963517905
90800000 27.48911885939877
90900000 28.593302063442756
91000000 29.07755100030127
91100000 29.109644771530853
91200000 29.251001484382854
91300000 30.297486701601585
91400000 30.892808904434155
91500000 29.53817052491871
91600000 28.97282515676981
91700000 29.75136953033188
91800000 28.371685413223748
91900000 28.762949918927596
92000000 13.657576443087335
92100000 21.995049141564355
92200000 26.814134428967265
92300000 26.80915011221862
92400000 25.40560877340295
92500000 26.309504147838215
92600000 26.86470090710068
92700000 4.513056267040464
92800000 27.548686026685747
92900000 26.538218255204242
93000000 25.19733932781002
93100000 26.664704050524325
93200000 25.

118700000 25.338175601089723
118800000 23.92593172836063
118900000 23.681240782280263
119000000 20.462596821546036
119100000 11.939507934864242
119200000 23.362861972843547
119300000 22.557838543930902
119400000 22.954960371319963
119500000 16.67265993486568
119600000 13.150505531879228
119700000 22.740052908637455
119800000 22.84990735280713
119900000 23.136260538921963
120000000 14.793845671626917
120100000 17.8578807798727
120200000 23.30655828633138
120300000 23.229076945552784
120400000 23.978850009372206
120500000 23.904569293982494
120600000 24.812376490258778
120700000 26.1585319844461
120800000 25.1434044849307
120900000 25.813449931181125
121000000 25.546601376189194
121100000 25.503112371953186
121200000 26.006210016071936
121300000 24.726210729090255
121400000 25.86061659940557
121500000 23.932546196000356
121600000 23.432731663868232
121700000 21.065637029034715
121800000 9.683995493472922
121900000 23.30887897006606
122000000 23.89166950177615
122100000 22.530583031536295

...

233400000 14.252881267210938
233500000 14.553400274933784
233600000 14.43353754655737
233700000 13.933017135565951
233800000 14.281604558185897
233900000 13.920595134906906
234000000 15.015824311143428
234100000 13.701428875300406
234200000 13.765787226819524
234300000 13.021008977347764
234400000 13.280501057730898
234500000 12.879233931589118
234600000 13.417329004854473
234700000 4.711522154762806
234800000 14.134836895407183
234900000 12.370435671818106
235000000 7.134680121088652
235100000 9.178848730831245
235200000 14.429776533262443
235300000 12.108115044218872
235400000 4.26370529643126
235500000 12.750785347024992
235600000 13.783310941653996
235700000 12.77184229915066
235800000 14.10887629418645
235900000 13.124605510982441
236000000 14.354744684598753
236100000 14.734116189418076
236200000 15.268003671322806
236300000 14.532993587633973
236400000 14.109623047699065
236500000 14.365076859592781
236600000 14.93046055370236
236700000 14.477039005410953
236800000 14.0062327236

262000000 11.418110209446706
262100000 10.799597870017005
262200000 10.77990293011147
262300000 10.649649209288375
262400000 6.3094579520080805
262500000 7.522659654069325
262600000 11.362634879132251
262700000 11.045085821961523
262800000 11.892421043853842
262900000 11.80901742766437
263000000 11.96851463358465
263100000 11.314129138812309
263200000 12.35770920467389
263300000 10.982869037885258
263400000 12.565511360825763
263500000 10.500231349549221
263600000 12.137254087727547
263700000 10.574501714153302
263800000 11.751660582245789
263900000 9.851638396615309
264000000 5.266833672862229
264100000 9.056304966552347
264200000 10.853593717436723
264300000 9.943872698038387
264400000 10.67257168221717
264500000 9.377841574310052
264600000 3.2571086179409385
264700000 10.51357428721654
264800000 10.55130526374108
264900000 10.221707762507673
265000000 10.242444077957948
265100000 6.337789283523655
265200000 7.140273206354131
265300000 11.238554070021912
265400000 11.117092992513548


291100000 7.951486681438679
291200000 8.536900144203225
291300000 7.946222028822066
291400000 4.4766082062686685
291500000 6.155440896815264
291600000 8.526065649950075
291700000 7.913045132275261
291800000 9.174010809479858
291900000 7.69874641750328
292000000 9.607574740223399
292100000 8.533215160780156
292200000 2.711916962507406
292300000 8.530659571007245
292400000 10.067083678018284
292500000 9.228804150388934
292600000 9.72456494639911
292700000 9.463033871667571
292800000 9.711176266708007
292900000 8.985007390394678
293000000 9.723948728978108
293100000 9.290884443209295
293200000 8.997974029887745
293300000 5.929097395688062
293400000 5.405280889678356
293500000 8.906043274182418
293600000 8.535072018379536
293700000 8.18370688744757
293800000 8.314323628346775
293900000 7.926456355669907
294000000 8.145071187536564
294100000 3.1027430658470063
294200000 7.3010508571827835
294300000 7.89577240112384
294400000 8.176747251257904
294500000 9.267594646731371
294600000 7.58167637

320300000 5.926203149243873
320400000 5.952509728082126
320500000 5.548117806662461
320600000 6.227088781419275
320700000 5.107490430607356
320800000 5.892141516543501
320900000 4.464059268484569
321000000 1.8384476834357573
321100000 5.451248765643458
321200000 5.697716323334355
321300000 5.348297001097171
321400000 6.288559313672974
321500000 5.826191016749789
321600000 5.846894764979753
321700000 6.16689810278258
321800000 6.152055684758463
321900000 3.6976976373642327
322000000 3.787318780169107
322100000 6.134062733893668
322200000 6.104778187750538
322300000 5.874318935031467
322400000 5.986511855525867
322500000 6.113116722360797
322600000 4.867470547445158
322700000 2.981455858600598
322800000 5.294617572996297
322900000 6.320909974908207
323000000 5.530521842647571
323100000 5.68600655100717
323200000 5.306337550649181
323300000 5.797917183453787
323400000 4.906918296667375
323500000 5.632628375983324
323600000 3.689943194122257
323700000 2.5889209199081247
323800000 5.1083587

349300000 1.729387444602634
349400000 2.2170549216024917
349500000 2.7250396722883585
349600000 3.0595673344402825
349700000 2.800061004215856
349800000 3.009456826178628
349900000 2.674143023178222
350000000 2.96467357546252
350100000 2.784358268067759
350200000 2.9433316341943025
350300000 2.6243710509876452
350400000 2.693881974182316
350500000 0.9179212345210128
350600000 2.37291991162288
350700000 2.772622839584222
350800000 2.6651104426867858
350900000 2.9604249667932034
351000000 2.6355418137842754
351100000 2.763531213717672
351200000 2.7603877425157606
351300000 2.8988532820784165
351400000 2.599169553620718
351500000 2.9767292154100957
351600000 2.8387418748115034
351700000 0.5795348915094111
351800000 2.6999581501119554
351900000 2.745639299857141
352000000 0.8848589051418795
352100000 2.712474102949295
352200000 2.627855978874966
352300000 2.4913500825259214
352400000 2.60646127640989
352500000 2.676486638891381
352600000 2.337679571364617
352700000 2.598669434327865
352800